## Import Libraries

In [67]:
import glob
import os

import pandas as pd
import numpy as np
from numpy import zeros, newaxis
import cv2

import matplotlib.pyplot as plt
import seaborn

## Read Data

In [68]:
axial_image = 'MRNet-v1.0/train/axial/0000.npy'
coronal_image = 'MRNet-v1.0/train/coronal/0000.npy'
saggital_image = 'MRNet-v1.0/train/sagittal/0000.npy'


img_array_axial = np.load(axial_image)
img_array_coronal = np.load(coronal_image)
img_array_sagittal = np.load(saggital_image)

In [69]:
abnormal_path_train =  'MRNet-v1.0/train-abnormal.csv'
acl_path_train = 'MRNet-v1.0/train-acl.csv'
meniscus_train =  'MRNet-v1.0/train-meniscus.csv'


abnormal_path_valid =  'MRNet-v1.0/valid-abnormal.csv'
acl_path_valid = 'MRNet-v1.0/valid-acl.csv'
meniscus_valid =  'MRNet-v1.0/valid-meniscus.csv'


In [70]:
def load_labels(abnormal_path, acl_path, meniscus_path):
    
    df_labels_abnormal =  pd.read_csv(abnormal_path, names=['Image_Name', 'Abnormal_Label'])
    df_labels_acl = pd.read_csv(acl_path, names=['Image_Name', 'Acl_Label'])
    df_labels_meniscus = pd.read_csv(meniscus_path, names=['Image_Name', 'Meniscus_Label'])
    
    df_lables_combined = pd.merge(df_labels_abnormal, df_labels_acl, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined = pd.merge(df_lables_combined, df_labels_meniscus, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].astype(str)
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].apply(lambda x: x.zfill(4))
    
    return df_lables_combined

In [71]:
# Load Train Labels
df_labels_train = load_labels(abnormal_path_train, acl_path_train, meniscus_train)
df_labels_train

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,0000,1,0,0
1,0001,1,1,1
2,0002,1,0,0
3,0003,1,0,1
4,0004,1,0,0
...,...,...,...,...
1125,1125,1,0,1
1126,1126,1,0,1
1127,1127,0,0,0
1128,1128,1,0,0


In [72]:
# Load Valid Labels
df_labels_valid = load_labels(abnormal_path_valid, acl_path_valid, meniscus_valid)
df_labels_valid

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,1130,0,0,0
1,1131,0,0,0
2,1132,0,0,0
3,1133,0,0,0
4,1134,0,0,0
...,...,...,...,...
115,1245,1,1,1
116,1246,1,1,1
117,1247,1,0,1
118,1248,1,1,1


In [73]:
glob.glob('MRNet-v1.0\*')

['MRNet-v1.0\\train',
 'MRNet-v1.0\\train-abnormal.csv',
 'MRNet-v1.0\\train-acl.csv',
 'MRNet-v1.0\\train-meniscus.csv',
 'MRNet-v1.0\\valid',
 'MRNet-v1.0\\valid-abnormal.csv',
 'MRNet-v1.0\\valid-acl.csv',
 'MRNet-v1.0\\valid-meniscus.csv']

In [74]:
# Read Traning Data
def read_data(train_path, df_labels):
    axial_images = []
    coronal_images = []
    sagittal_images = []

    axial_labels = []
    coronal_labels = []
    sagittal_labels = []

    # train_path = '/kaggle/input/mrnet-v1/MRNet-v1.0/train'
    train_folders = glob.glob(train_path + '/*')
    
    for folder in train_folders:
        train_data = glob.glob(folder + '/*.npy')
        print('Total samples in data:', len(train_data))
        print('Loading Data From :   ', folder)
        
        for i in train_data:
#             print(i)
            img_array = np.load(i)
            start, end = (img_array.shape[0]/2)-8, (img_array.shape[0]/2)+8
            image_tensor = img_array[int(start):int(end),:,:].reshape(256,256,16)
            if 'axial' in i:
                axial_images.append(image_tensor)
                image_name = i.rsplit('\\')[-1].split('.')[0]
                axial_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])
            if 'coronal' in i:
                coronal_images.append(image_tensor)
                image_name = i.rsplit('\\')[-1].split('.')[0]
                coronal_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])   
            if 'sagittal' in i:
                sagittal_images.append(image_tensor)
                image_name = i.rsplit('\\')[-1].split('.')[0]
                sagittal_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])

        print('Loading Complete From :   ', folder)
        print('\n\n')
    return axial_images, coronal_images, sagittal_images, axial_labels

train_path =  'MRNet-v1.0\\train'
valid_path =  'MRNet-v1.0\\valid'

axial_images_train, coronal_images_train, sagittal_images_train, axial_labels_train = read_data(train_path, df_labels_train)

Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\axial
Loading Complete From :    MRNet-v1.0\train\axial



Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\coronal
Loading Complete From :    MRNet-v1.0\train\coronal



Total samples in data: 1130
Loading Data From :    MRNet-v1.0\train\sagittal
Loading Complete From :    MRNet-v1.0\train\sagittal





In [75]:
#As the the dimensions and memory of the data exceeed the Kaggle's allowed Limit,
# So we Dropped the Axial Images and Train Model on the base of Coronal and Sagittal Images.

del axial_images_train

In [76]:
# print(np.array(axial_images_train).shape)
print(np.array(coronal_images_train).shape)
print(np.array(sagittal_images_train).shape)
print(np.array(axial_labels_train).shape)

(1130, 256, 256, 16)
(1130, 256, 256, 16)
(1130, 3)


## Modeling

In [77]:
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K


In [78]:
# Two Input Model

def model ():
    visible1 = Input(shape=(256,256,16))
    
    conv11 = Conv2D( filters=16, kernel_size=(5,5), activation="relu")(visible1)
    pool11 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv11)
    
    conv12 = Conv2D( filters=16, kernel_size=(5,5), activation="relu")(pool11)
    pool12 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv12)
    
    conv13 = Conv2D(filters=16, kernel_size=(5,5), activation='relu')(pool12)
    pool13 = MaxPooling2D(pool_size=(2, 2),strides=(2,2))(conv13)
    flat1 = Flatten()(pool13)
    
    return flat1, visible1
model()

# merge input models
model_1, Visible_1 = model()
model_2, Visible_2 = model()

merge = concatenate([model_1, model_2])

hidden1 = Dense(10, activation='relu')(merge)
hidden2 = Dense(5, activation='relu')(hidden1)
output = Dense(3, activation='sigmoid')(hidden2)
model = Model(inputs=[Visible_1, Visible_2], outputs=output)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 input_3 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 conv2d_3 (Conv2D)              (None, 252, 252, 16  6416        ['input_2[0][0]']                
                                )                                                             

In [79]:
model.compile(loss=tensorflow.keras.losses.binary_crossentropy,
              optimizer=tensorflow.keras.optimizers.SGD( learning_rate = 0.001, momentum = 0.9),metrics=["accuracy"])

# Give Class Weights to Balance Results
class_weight = {0: 1.,
                1: 50.,
                2: 50.}

model.fit(x = [np.array(coronal_images_train), np.array(sagittal_images_train)],
          y = np.array(axial_labels_train),
          epochs=1,
          batch_size=32,
          validation_split=0.20,
          class_weight=class_weight)

# model.save_weights('model_3_Input.h5')

Epoch 1/5
29/29 [==============================] - 292s 10s/step - loss: 1.0094 - accuracy: 0.9292 - val_loss: 0.6884 - val_accuracy: 1.0000
Epoch 2/5
29/29 [==============================] - 267s 9s/step - loss: 0.6855 - accuracy: 1.0000 - val_loss: 0.6817 - val_accuracy: 1.0000
Epoch 3/5
29/29 [==============================] - 295s 10s/step - loss: 0.6791 - accuracy: 1.0000 - val_loss: 0.6755 - val_accuracy: 1.0000
Epoch 4/5
29/29 [==============================] - 296s 10s/step - loss: 0.6730 - accuracy: 1.0000 - val_loss: 0.6693 - val_accuracy: 1.0000
Epoch 5/5
29/29 [==============================] - 231s 8s/step - loss: 0.6671 - accuracy: 1.0000 - val_loss: 0.6634 - val_accuracy: 1.0000


In [80]:
# from keras.utils.vis_utils import plot_model
# plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)


In [81]:
del coronal_images_train, sagittal_images_train

In [82]:
axial_images_valid, coronal_images_valid, sagittal_images_valid, axial_labels_valid = read_data(valid_path, df_labels_valid)

Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\axial
Loading Complete From :    MRNet-v1.0\valid\axial



Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\coronal
Loading Complete From :    MRNet-v1.0\valid\coronal



Total samples in data: 120
Loading Data From :    MRNet-v1.0\valid\sagittal
Loading Complete From :    MRNet-v1.0\valid\sagittal





In [83]:
del axial_images_valid

## Evaluate Model

In [84]:
y_pred = model.predict([ np.array(coronal_images_valid), np.array(sagittal_images_valid)])

4/4 [==============================] - 10s 2s/step


In [85]:
y_pred

array([[0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.

In [86]:
from sklearn.metrics import classification_report
# 
print(classification_report(y_pred.round(), axial_labels_valid))

              precision    recall  f1-score   support

           0       1.00      0.79      0.88       120
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0

   micro avg       0.47      0.79      0.59       120
   macro avg       0.33      0.26      0.29       120
weighted avg       1.00      0.79      0.88       120
 samples avg       0.44      0.79      0.54       120



c:\Users\jctub\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\jctub\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [91]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(y_pred.round(), axial_labels_valid))

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [90]:
y_pred

array([[0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.53301126, 0.46640822, 0.48428705],
       [0.

In [92]:
axial_labels_valid

[array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([0, 0, 0], dtype=int64),
 array([1, 0, 0], dtype=int64),
 array([1, 0, 0], dtype=int64),
 array([1, 0, 0], dtype=int64),
 array([1, 0, 0], dtype=int64),
 array([1, 0, 0], dtype=int64),
 array([1, 0, 0], dtype=int64),
 array([1, 0, 0], dtype=int64),
 array([